In [3]:
import cv2
import scipy.stats as ss
from mpmath import *
import numpy as np
import matplotlib.pyplot as plt
import math
%xmode plain

Exception reporting mode: Plain


In [4]:
def read_img(img_name='train.jpg', mask_name='mask.jpg'):
    
    img = cv2.imread(img_name, 1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    mask = cv2.imread(mask_name,1)
    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    
    return np.array(img).astype(float), np.array(mask).astype(float)

In [5]:
img, mask = read_img()

In [6]:
def get_pixels(img, mask, num_pixels):
    
    if num_pixels:
        ncoin = np.sum(mask > 200 )
        ndesk = np.sum(mask <= 200 )
        return ncoin, ndesk
    else:
        coin = img[(mask > 200).nonzero()]
        desk = img[(mask <= 200).nonzero()]
        return coin, desk

In [7]:
def ML(img, mask, percentage):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl = coin_pxl[:,2]
    desk_pxl = desk_pxl[:,2]
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))
    
    # sigma calculation functions
    calc_mu =  lambda x: np.mean(x)
    calc_sigma = lambda x, mu: np.mean((x-mu)**2)
    
    # coin paramaters
    coin_mu = calc_mu(coin_pxl[0:num_elements_coin])
    coin_sig = calc_sigma(coin_pxl[0:num_elements_coin], coin_mu)

    # non-coin paramaters
    desk_mu = calc_mu(desk_pxl[0:num_elements_desk])
    desk_sig = calc_sigma(desk_pxl[0:num_elements_desk], desk_mu)
    
    print("{:.2f}% mu coin {:.6f} sigma coin {:.6f} ____ mu coin {:.6f} sigma coin {:.6f}".format(percentage, coin_mu, coin_sig, desk_mu, desk_sig))
    
    return coin_mu, coin_sig, desk_mu, desk_sig

def get_prob(pixels, c_mu, c_var, d_mu, d_var):
    
    p1 = ss.norm.pdf(pixels, c_mu, np.sqrt(c_var))
    p2 = ss.norm.pdf(pixels, d_mu, np.sqrt(d_var))
    print(p1)
    return p1/(p1 + p2)

def test_model(img, mask, c_mu, c_var, d_mu, d_var, eps=1):
    
    img_reshape = img[:,:, 2].flatten()
    mask_reshape = mask.flatten()
    
    chunk_size =int(np.floor(eps*len(img_reshape)))
    pxl = img_reshape[0:chunk_size]
    mask_pxl = mask_reshape[0:chunk_size]
    x = get_prob(pxl, c_mu, c_var, d_mu, d_var)
    
    x[ x > 0.5] = 1
    x[ x <= 0.5] = 0
    mask_pxl[ mask_pxl > 200] = 1
    mask_pxl[ mask_pxl <= 200] = 0
    
    return np.sum(x == mask_pxl)/chunk_size
    

In [8]:
coin_mu, coin_sig, desk_mu, desk_sig = ML(img, mask, 0.5)
test_model(img, mask, coin_mu, coin_sig, desk_mu, desk_sig, eps=0.5)

0.50% mu coin 207.997043 sigma coin 1308.684644 ____ mu coin 127.514487 sigma coin 135.641760
[0.00107911 0.00121386 0.00128594 ... 0.00089933 0.00114494 0.00152192]


0.9130143229166666

In [9]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
print("validation image 1: ", test_model(img1, mask1, coin_mu, coin_sig, desk_mu, desk_sig, eps=0.5))
###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
print("validation image 2: ", test_model(img2, img2, coin_mu, coin_sig, desk_mu, desk_sig, eps=0.5))

[0.00065369 0.00061189 0.00053489 ... 0.00061189 0.00069783 0.00074437]
validation image 1:  0.9481022135416667
[0.00057231 0.00053489 0.00053489 ... 0.00049953 0.00069783 0.00089933]
validation image 2:  0.9702408854166666


In [10]:
def MAP(img, mask, percentage):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl = coin_pxl[:,2]
    desk_pxl = desk_pxl[:,2]
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))    
    
    #sigma calculation functions
    calc_mu = lambda x: (np.sum(x))/(len(x)+1)
    calc_sigma = lambda x, mu: (np.sum((x-mu)**2) + 2 + (-mu)**2)/(len(x) + 5)
    
    # coin paramaters
    coin_mu = calc_mu(coin_pxl[0:num_elements_coin])
    coin_sig = calc_sigma(coin_pxl[0:num_elements_coin], coin_mu)

    # non-coin paramaters
    desk_mu = calc_mu(desk_pxl[0:num_elements_desk])
    desk_sig = calc_sigma(desk_pxl[0:num_elements_desk], desk_mu)
    
    print("{:.2f}% mu coin {:.6f} sigma coin {:.6f} ____ mu coin {:.6f} sigma coin {:.6f}".format(percentage, coin_mu, coin_sig, desk_mu, desk_sig))
    
    return coin_mu, coin_sig, desk_mu, desk_sig

In [11]:
coin_mu1, coin_sig1, desk_mu1, desk_sig1 = MAP(img, mask, 0.5)
test_model(img, mask, coin_mu1, coin_sig1, desk_mu1, desk_sig1, eps=0.5)

0.50% mu coin 207.988129 sigma coin 1310.258022 ____ mu coin 127.514272 sigma coin 135.668125
[0.00108205 0.00121698 0.00128915 ... 0.000902   0.00114797 0.00152538]


0.9130143229166666

In [12]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
print("validation image 1: ", test_model(img1, mask1, coin_mu1, coin_sig1, desk_mu1, desk_sig1, eps=0.5))
###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
print("validation image 2: ", test_model(img2, img2, coin_mu1, coin_sig1, desk_mu1, desk_sig1, eps=0.5))

[0.00065591 0.00061401 0.00053684 ... 0.00061401 0.00070013 0.00074676]
validation image 1:  0.9481022135416667
[0.00057435 0.00053684 0.00053684 ... 0.0005014  0.00070013 0.000902  ]
validation image 2:  0.9702408854166666


## BA(Bayesian Approach)

In [25]:
def BA_tilde_vars(img_train, mask_train, percentage, coin):
        
    coin_pxl, desk_pxl = get_pixels(img_train, mask_train, False)
    coin_pxl = coin_pxl[:, 2]
    desk_pxl = desk_pxl[:, 2]

    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))   
    
    coin_pxl = coin_pxl[0:num_elements_coin]
    desk_pxl = desk_pxl[0:num_elements_desk]
    
    if coin:
        # coin 
        alpha_tc = 1 + len(coin_pxl)/2
        gamma_tc = 1 + len(coin_pxl)
        delta_tc = np.sum(coin_pxl)/(1 + len(coin_pxl))
        beta_tc = np.sum(coin_pxl**2)/2 + 1 - (np.sum(coin_pxl)**2)/(2*(1+ len(coin_pxl)))
        
        return alpha_tc, gamma_tc, delta_tc, beta_tc
    else:
        # desk  
        alpha_td = 1 + len(desk_pxl)/2
        gamma_td = 1 + len(desk_pxl)
        delta_td = np.sum(desk_pxl)/(1 + len(desk_pxl))
        beta_td = np.sum(desk_pxl**2)/2 + 1 - (np.sum(desk_pxl))**2/(2*(1+ len(desk_pxl)))
        
        return alpha_td, gamma_td, delta_td, beta_td
    
def BA_test(img_train, mask_train, percentage, img_val, mask_val):
    
    coin_pxl, desk_pxl = get_pixels(img_val, mask_val, False)
    coin_pxl = coin_pxl[:, 2]
    desk_pxl = desk_pxl[:, 2]
 
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))   
    
    coin_pxl = coin_pxl[0:num_elements_coin]
    desk_pxl = desk_pxl[0:num_elements_desk]
    
    alpha_tc, gamma_tc, delta_tc, beta_tc = BA_tilde_vars(img_train, mask_train, percentage, True)
    alpha_td, gamma_td, delta_td, beta_td = BA_tilde_vars(img_train, mask_train, percentage, False)
    
    test_img = img_val[:,:,2]
    test_img = test_img.flatten()
    test_img = test_img[0:int(np.floor(percentage*len(test_img)))]  
    
    # coin
    alpha_kc = alpha_tc + 0.5
    gamma_kc = gamma_tc + 1
    beta_kc = (test_img**2)/2 + beta_tc + (gamma_tc*(delta_tc**2))/2 - ((gamma_tc*delta_tc + test_img)**2)/(2*(gamma_tc + 1))

    ba_pdf = lambda alpha_t, gamma_t, beta_t, alpha_k, beta_k, gamma_k: ((1/sqrt(2*pi)) * (sqrt(gamma_t)*power(beta_t, alpha_t))/(sqrt(gamma_k)*(power(beta_k, alpha_k)))) * gamma(alpha_k)/gamma(alpha_t)
 
    p1 = np.array([ba_pdf(alpha_tc, gamma_tc, beta_tc, alpha_kc, i, gamma_kc ) for i in beta_kc ])
#     p1 = ba_pdf(alpha_tc, gamma_tc, beta_tc, alpha_kc, matrix(beta_kc*mpf(1.0)), gamma_kc ) 
    alpha_kd = alpha_td + 0.5
    gamma_kd = gamma_td + 1
    beta_kd = (test_img**2)/2 + beta_td + (gamma_td*(delta_td**2))/2 - ((gamma_td*delta_td + test_img)**2)/(2*(gamma_td + 1))
    print(beta_kd)
    p2 = np.array([ba_pdf(alpha_td, gamma_td, beta_td, alpha_kd, i, gamma_kd ) for i in beta_kd ])
#     p2 = ba_pdf(alpha_td, gamma_td, beta_td, alpha_kd, matrix(beta_kd*mpf(1.0)), gamma_kd )
    
    mask_reshape = mask_val.flatten()
    chunk_size =int(np.floor(percentage*len(mask_reshape)))
    mask_pxl = mask_reshape[0:chunk_size]
    
    x = p1/(p1 + p2)
#     x = np.array(x.tolist(), dtype=float)
    x[ x > 0.5] = 1
    x[ x <= 0.5] = 0
    mask_pxl[ mask_pxl > 200] = 1
    mask_pxl[ mask_pxl <= 200] = 0
    
    return np.sum(x == mask_pxl)/chunk_size
    
    

In [26]:
probs = BA_test(img, mask,  0.25, img1, mask1)
display(probs)

[18873098.01818037 18873101.37544441 18873111.08996248 ...
 18873253.37678432 18873271.73399782 18873271.73399782]


0.98640625